### Preparación de los datos

In [2]:
import os
import re
from tqdm import tqdm
from glob import glob

import pandas as pd
import numpy as np

import cv2
from sklearn.model_selection import train_test_split

In [ ]:
# Directorio donde se encuentran las imágenes
IMAGE_PATH = "data/UTKFACE"

# Regex para capturar las etiquetas de género en el nombre del archivo
# [age]_[gender]_[race]_[date&time].jpg.chip.jpg
FILENAME_PATTERN = re.compile(r"\d+_([01])_[0-4]_\d+.jpg.chip.jpg")

In [4]:
data = []
invalid_file_paths = []

# Filtrar imágenes con etiquetas válidas
try:
    all_files = glob(f"{IMAGE_PATH}/*.jpg")
    if not all_files:
        raise FileNotFoundError(
            f"No se encontraron archivos en el directorio {IMAGE_PATH}"
        )

    for file_path in tqdm(all_files, desc="Filtrando imágenes"):
        filename = os.path.basename(file_path)
        match = FILENAME_PATTERN.match(filename)

        if match:
            [gender] = match.groups()
            data.append(
                {
                    "file_path": filename,
                    "gender": int(gender),
                }
            )
        else:
            invalid_file_paths.append(filename)

    df = pd.DataFrame(data)
    print("\n¡Filtrado completado con éxito!")

except FileNotFoundError as e:
    print(f"\nError de ruta: {e}")
    df = pd.DataFrame()
except Exception as e:
    print(f"\nError inesperado: {e}")
    df = pd.DataFrame()

if invalid_file_paths:
    print(f"\nImágenes con etiqueta inválida: {len(invalid_file_paths)}")
    for file in invalid_file_paths:
        print(f"- {file}")
else:
    print("\nSe filtraron todas las imágenes correctamente.")

Filtrando imágenes: 100%|██████████| 23708/23708 [00:00<00:00, 214842.03it/s]


¡Filtrado completado con éxito!

Imágenes con etiqueta inválida: 4
- 24_0_1_20170116220224657 .jpg.chip.jpg
- 39_1_20170116174525125.jpg.chip.jpg
- 61_1_20170109142408075.jpg.chip.jpg
- 61_1_20170109150557335.jpg.chip.jpg


In [5]:
# Dimensión objetivo en píxeles
TARGET_SIZE = (32, 32)


def preprocess_image(filename: str, images_path: str) -> np.ndarray:
    """
    Preprocesa una imagen: redimensiona, convierte a escala de grises y normaliza.

    Args:
        filename: Nombre del archivo de la imagen.
        images_path: Ruta del directorio que contiene las imágenes.

    Returns:
        Vector numpy con la imagen preprocesada.
    """
    file_path = os.path.join(images_path, filename)

    try:
        img = cv2.imread(file_path)

        if img is None:
            raise ValueError(f"Error al cargar la imagen: {file_path}")

        # Redimensionar a tamaño objetivo
        img = cv2.resize(img, TARGET_SIZE)
        # Convertir a escala de grises
        img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        # Normalizar a rango [0, 1]
        img = img / 255.0

        # Aplanar la imagen en un vector
        vector = img.flatten()

        return vector

    except ValueError as e:
        print(e)
        return None
    except Exception as e:
        print(f"Error inesperado al procesar la imagen: {e}")
        return None

In [22]:
image_vectors = []
valid_indices = []
invalid_images = []

# Preprocesar imágenes y almacenar vectores válidos
for index, row in tqdm(df.iterrows(), total=len(df), desc="Preprocesando imágenes"):
    vector = preprocess_image(row["file_path"], IMAGE_PATH)

    if vector is not None:
        image_vectors.append(vector)
        valid_indices.append(index)
    else:
        invalid_images.append(row["file_path"])

print(f"\n¡Preprocesamiento completado!")

if invalid_images:
    print(f"\nImágenes no procesadas: {len(invalid_images)}")
    for img in invalid_images:
        print(f"- {img}")
else:
    print("\nSe procesaron todas las imágenes correctamente.")

Preprocesando imágenes: 100%|██████████| 23704/23704 [00:11<00:00, 2049.32it/s]


¡Preprocesamiento completado!

Se procesaron todas las imágenes correctamente.


In [23]:
X = np.array(image_vectors)

df = df.loc[valid_indices].reset_index(drop=True)
y = df["gender"].values

print("¡Datos preparados!")
print(f"\nDimensión de la matriz de datos X: {X.shape}")
print(f"Dimensión del vector de etiquetas y: {y.shape}")

# Borrar variables temporales para liberar memoria
del image_vectors, valid_indices

¡Datos preparados!

Dimensión de la matriz de datos X: (23704, 1024)
Dimensión del vector de etiquetas y: (23704,)


### División del conjunto de datos

In [11]:
TEST_SIZE = 0.2
VALIDATION_SIZE = 0.2

RANDOM_SEED = 42

In [19]:
# Separar el conjunto de prueba del resto de los datos
X_temp, X_test, y_temp, y_test = train_test_split(
    X, y, test_size=TEST_SIZE, random_state=RANDOM_SEED, stratify=y
)

# Separar el conjunto de validación del conjunto de entrenamiento
VALIDATION_SIZE_ADJUSTED = VALIDATION_SIZE / (1 - TEST_SIZE)

X_train, X_val, y_train, y_val = train_test_split(
    X_temp,
    y_temp,
    test_size=VALIDATION_SIZE_ADJUSTED,
    random_state=RANDOM_SEED,
    stratify=y_temp,
)

print("¡División de datos completada!")
print(f"\nDimensiones de los conjuntos:")
print(
    f"- Entrenamiento ({round((1 - TEST_SIZE - VALIDATION_SIZE)*100)}%): X_train: {X_train.shape}, y_train: {y_train.shape}"
)
print(
    f"- Validación ({round(VALIDATION_SIZE*100)}%): X_val: {X_val.shape}, y_val: {y_val.shape}"
)
print(
    f"- Prueba ({round(TEST_SIZE*100)}%): X_test: {X_test.shape}, y_test: {y_test.shape}"
)

print()


del X_temp, y_temp

¡División de datos completada!

Dimensiones de los conjuntos:
- Entrenamiento (60%): X_train: (14222, 1024), y_train: (14222,)
- Validación (20%): X_val: (4741, 1024), y_val: (4741,)
- Prueba (20%): X_test: (4741, 1024), y_test: (4741,)



### Análisis mediante PCA